<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/docs/examples/embeddings/gradient.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="在 Colab 中打开"/></a>


# 梯度嵌入

[Gradient](https://gradient.ai)提供了嵌入模型，可以很容易地与LlamaIndex集成。下面是如何在LlamaIndex中使用它的示例。


In [ ]:
%pip install llama-index-llms-gradient
%pip install llama-index-embeddings-gradient

In [ ]:
# 安装所需的包

%pip install llama-index --quiet
%pip install gradientai --quiet

Gradient需要访问令牌和工作空间ID进行授权。它们可以从以下位置获取：
- [Gradient UI](https://auth.gradient.ai/login)，或
- [Gradient CLI](https://docs.gradient.ai/docs/cli-quickstart) 使用 `gradient env` 命令。


In [ ]:
import os

os.environ["GRADIENT_ACCESS_TOKEN"] = "{GRADIENT_ACCESS_TOKEN}"
os.environ["GRADIENT_WORKSPACE_ID"] = "{GRADIENT_WORKSPACE_ID}"

In [ ]:
from llama_index.llms.gradient import GradientBaseModelLLM

# 注意：我们在这里使用一个基本模型，您也可以插入您微调过的模型。
llm = GradientBaseModelLLM(
    base_model_slug="llama2-7b-chat",
    max_tokens=400,
)

#### 下载数据


In [ ]:
!mkdir -p 'data/paul_graham/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt'

### 加载文档


In [ ]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader("./data/paul_graham").load_data()
print(f"Loaded {len(documents)} document(s).")

### 配置梯度嵌入

在使用深度学习模型时，梯度嵌入是一种有用的技术，它可以帮助我们理解模型的行为。在这个示例中，我们将学习如何配置梯度嵌入。


In [ ]:
from llama_index.embeddings.gradient import GradientEmbedding
from llama_index.core import Settings

embed_model = GradientEmbedding(
    gradient_access_token=os.environ["GRADIENT_ACCESS_TOKEN"],
    gradient_workspace_id=os.environ["GRADIENT_WORKSPACE_ID"],
    gradient_model_slug="bge-large",
)

Settings.embed_model = embed_model
Settings.llm = llm
Settings.chunk_size = 1024

### 设置查询索引


In [ ]:
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()

In [ ]:
response = query_engine.query(
    "What did the author do after his time at Y Combinator?"
)
print(response)